In [ ]:
import logging
import os
import uuid

from dapr_agents import OpenAIChatClient
from dapr_agents.tool import AgentTool
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

In [ ]:
logging.basicConfig(level=logging.INFO)
load_dotenv()

In [ ]:
llm = OpenAIChatClient(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [ ]:
NOTEBOOK_NAME = f"mcp_{uuid.uuid4().hex[:8]}.ipynb"

raw_env = {
    **os.environ,
    "MCP_JUPYTER_SESSION_MODE": "server",
    "MCP_JUPYTER_BASE_URL": os.getenv("JUPYTER_BASE_URL"),
    "MCP_JUPYTER_TOKEN": os.getenv("JUPYTER_TOKEN"),
    "MCP_JUPYTER_KERNEL_NAME": os.getenv("JUPYTER_KERNEL_NAME", "python3"),
    "MCP_JUPYTER_NOTEBOOK_PATH": os.getenv("JUPYTER_NOTEBOOK_PATH", NOTEBOOK_NAME),
    "MCP_JUPYTER_TOOLSETS": "postgresql",
}

ENV = {k: v for k, v in raw_env.items() if v is not None}


In [ ]:
# Adjust path to your MCP server entrypoint (where your server.py lives)
server_params = StdioServerParameters(
    command="uvx",
    #args=[ "mcp-jupyter-notebook", "--transport", "stdio"],
    args = ["--refresh", "mcp-jupyter-notebook", "--transport", "stdio"],
    #args=[
    #    "--refresh",
    #    "--from",
    #    "../../../",
    #    "mcp-notebook-data",
    #    "--transport",
    #    "stdio",
    #],
    env=ENV,
)

In [ ]:
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as s:
        await s.initialize()

        # Discover tools
        tools = await AgentTool.from_mcp_session(s)

        from dapr_agents import Agent

        notebook_agent = Agent(
            name="Jupyter Notebook Orchestrator",
            role=(
                "A notebook co-pilot that drives analysis inside a persistent Jupyter session, "
                "with read-only access to a PostgreSQL database via MCP tools."
            ),
            goal=(
                "Use the active Jupyter notebook to complete tasks end-to-end: "
                "add clear markdown, execute Python code, and, when necessary, install Python packages."
                "Reuse the same live session (kernel + document) so work builds on previous cells. "
                "When data is needed, discover Postgres schemas/tables first, then load results into DataFrames via the provided tools."
            ),
            instructions=[
                # Session & ordering
                "You are driving a live Jupyter notebook (persistent kernel + document).",
                "Cell order matters. Issue exactly ONE tool call at a time and wait for its result before the next.",
                "Append new cells; do not rewrite or reorder existing cells unless explicitly asked.",

                # PostgreSQL tools & workflow
                "Before querying data, inspect metadata using schema tools:",
                "Prefer small, targeted SQL with an explicit LIMIT while exploring.",
                "Never print or echo credentials/DSNs; do not read environment variables into the notebook output.",

                # Notebook hygiene
                "Before each task, add a short markdown heading describing what you'll do.",
                "Keep code cells small and focused—one idea per cell.",
                "When previewing data, show concise outputs (e.g., df.head(), shapes, key metrics).",
                "For Matplotlib plots, include a title and axis labels; add a legend when helpful.",

                # Robustness
                "If an error occurs, add a brief markdown note explaining the fix, then run a corrected code cell.",
                "Reuse variables and results created earlier; reference them directly rather than reloading the same data.",

                # Closure
                "End a logical section with a short markdown summary of what was produced and learned."
            ],
            llm=llm,
            tools=tools,
            max_iterations=20,
        )

        QUESTION = """
Connect to the Postgres database and start exploring.
1. List a few available schemas and tables.
2. Pick one small table that looks interesting.
3. Load a limited sample of rows into a pandas DataFrame so it's easy to work with.
4. Create a couple of simple visualizations (for example, a bar chart and a line chart) using Matplotlib.
As you go, add short markdown headings to explain each step, and finish with a brief summary of what you discovered.
"""
        await notebook_agent.run(QUESTION)